# Demo notebook showing arbok functionalities

This is a simple demo notebook containing the core functionalities of arbok

### Importing qm and arbok module

In [1]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.simulate.credentials import create_credentials
from qm import SimulationConfig

2023-09-12 14:00:27,850 - qm - INFO     - Starting session: aed4c32b-7620-4ccb-ac0a-844543d688b5


In [2]:
from arbok.core.sequence import Sequence
from arbok.core.program import Program
from arbok.core.sample import Sample

from arbok.samples.sunshine.readout.up_down_read import UpDownReadout
from arbok.samples.sunshine.initialization.mixed_down_up_init import (
    MixedDownUpInit
)
from arbok.samples.sunshine.control.smart_Y import SmartY
from arbok.samples.sunshine.control.smart_swap import SmartSwap
from arbok.samples.sunshine.control.dummy_control import Square
from arbok.samples.sunshine.configs.rf2v_config import rf2v_config

### Instanciating Arbok modules

Firstly we instanciate all `Sequence` objects that should build the entire 
measurement. Since we imported all sequences from the 'arbok.samples.sunshine' 
they already have the config for that specific sample

In [3]:
smart_Y = SmartY('smartY')
smart_swap12 = SmartSwap('smartSwap12')
smart_swap23 = SmartSwap('smartSwap32')
square = Square('square')
duInit = MixedDownUpInit('duInit')
stReadout = UpDownReadout('stReadout')

Next we create an empty `Sequence` class we call 'MEAS' that will structure all 
subsequeces. This time we need to pass a `Sample` since its a unconfigured 
sequence. We create second empty sequence to summarize all control sequences 
(everything happening before readout and after initialization). This is not
mandatory but is helpfull if you want to simulate the entire qubit control part
of the experiment.

In [4]:
sunshine = Sample('sunshine', rf2v_config)

In [5]:
MEAS = Program('MEAS', sample = sunshine)
MEAS.stream_mode = "pause_each"

In [6]:
MEAS.connect_OPX(host_ip= '192.168.0.3')

2023-09-12 14:00:28,494 - qm - INFO     - Performing health check
2023-09-12 14:00:28,506 - qm - INFO     - Health check passed
2023-09-12 14:00:29,244 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing existing identical Quantum Machine qm-1694490019569
2023-09-12 14:00:29,246 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1694490019569 is canceling running job 1684735461981.


In [7]:
CYY = Sequence('CYY', sample = sunshine)

Thus we add all sub-sequences responsible for qubit control in 'CYY' ...

In [8]:
CYY.add_subsequence(smart_swap12)
CYY.add_subsequence(smart_Y)
CYY.add_subsequence(smart_swap23)

... and finally add initialization, control and readout to the summarizing 
`Sequence`. 

In [9]:
MEAS.add_subsequence(duInit)
MEAS.add_subsequence(CYY)
MEAS.add_subsequence(square)
MEAS.add_subsequence(stReadout)

In [10]:
amp_range =  np.arange(0.01, 1.01, 0.01)
time_range =  np.arange(4, 104, 10, dtype=int)
print( len(amp_range) , len(time_range))

100 10


In [11]:
MEAS.settables = [MEAS.square.amp, MEAS.square.tSquare]
#MEAS.settables = [MEAS.square.tSquare]
MEAS.setpoints_grid = [ amp_range , time_range]

In [12]:
MEAS.run()

STREAMING
2023-09-12 14:00:29,940 - qm - INFO     - Sending program to QOP for compilation
2023-09-12 14:00:31,665 - qm - INFO     - Executing program


In [13]:
MEAS.qm_job.is_paused()

True

In [14]:
MEAS.qm_job.result_handles.diff_state.count_so_far()

-1

In [15]:
MEAS.stReadout.gettables

{'ref2_read_I': <arbok.core.sequence_parameter.GettableParameter at 0x286411d1750>,
 'ref2_read_Q': <arbok.core.sequence_parameter.GettableParameter at 0x28640f21150>,
 'ref2_read': <arbok.core.sequence_parameter.GettableParameter at 0x286411d1850>,
 'ref2_chopRef': <arbok.core.sequence_parameter.GettableParameter at 0x286411d1910>,
 'ref2_state': <arbok.core.sequence_parameter.GettableParameter at 0x286411d1990>,
 'read_read_I': <arbok.core.sequence_parameter.GettableParameter at 0x286411d1a10>,
 'read_read_Q': <arbok.core.sequence_parameter.GettableParameter at 0x28640cd1790>,
 'read_read': <arbok.core.sequence_parameter.GettableParameter at 0x286411d1ad0>,
 'read_chopRef': <arbok.core.sequence_parameter.GettableParameter at 0x286411d1b50>,
 'read_state': <arbok.core.sequence_parameter.GettableParameter at 0x28640cd1650>,
 'diff_read_I': <arbok.core.sequence_parameter.GettableParameter at 0x286411d1c10>,
 'diff_read_Q': <arbok.core.sequence_parameter.GettableParameter at 0x286411d1c9

In [16]:
#while MEAS.result_handles.is_processing():
    #print(MEAS.stReadout.diff_state.get())
    #print(MEAS.stReadout.ref2_read_I.get())

### Now putting everything in quantify

In [17]:
import quantify_core.data.handling as dh
from quantify_core.measurement import MeasurementControl
from quantify_core.visualization.pyqt_plotmon import PlotMonitor_pyqt

In [18]:
#dh.set_datadir(dh.default_datadir())
dh.set_datadir()

Data will be saved in:
C:\Users\labuser\quantify-data


In [19]:
meas_ctrl = MeasurementControl('meas_ctrl')

In [20]:
plotmon = PlotMonitor_pyqt("plotmon")
meas_ctrl.instr_plotmon(plotmon.name)

C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots is deprecated. Use "qcodes_loop.plots" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:18: QCoDeSDeprecationWarning: The qcodes.plots.colors is deprecated. Use "qcodes_loop.plots.colors" as an alternative.
  from qcodes.plots.colors import color_cycle
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\qcodes\plots\pyqtgraph.py:17: QCoDeSDeprecationWarning: The qcodes.utils.qt_helpers module is deprecated. Use "qcodes_loop.utils.qt_helpers" as an alternative.
  import qcodes.utils.qt_helpers
C:\Users\labuser\miniconda3\envs\temp2_env\Lib\site-packages\quantify_core\visualization\pyqt_plotmon_remote.py:19: QCoDeSDeprecationWarning: The qcodes.plots.pyqtgraph is deprecated. Use "qcodes_loop.plots

In [21]:
meas_ctrl.settables([MEAS.square.amp, MEAS.square.tSquare])
meas_ctrl.setpoints_grid([amp_range, time_range])
meas_ctrl.gettables(MEAS.stReadout.ref2_read_I)

In [22]:
MEAS.stReadout.diff_state.batch_size = 1000
MEAS.square.amp.batch_size = 1000
MEAS.square.tSquare.batch_size = 1000

In [23]:
MEAS.stReadout.diff_state.batch_size

1000

In [24]:
meas_ctrl.run("avg3", soft_avg=100)

Starting batched measurement...
Iterative settable(s) [outer loop(s)]:
	 --- (None) --- 
Batched settable(s):
	 amp, tSquare 
Batch size limit: 0

2023-09-12 14:00:37,244 - qm - WARNING  - Fetching single result will always return the single value
  1% completed | elapsed time:      2s | time left:    271s  last batch size:      0  2023-09-12 14:00:39,838 - qm - WARNING  - Fetching single result will always return the single value
  2% completed | elapsed time:      5s | time left:    261s  last batch size:      0  2023-09-12 14:00:42,505 - qm - WARNING  - Fetching single result will always return the single value
  3% completed | elapsed time:      8s | time left:    258s  last batch size:      0  2023-09-12 14:00:45,144 - qm - WARNING  - Fetching single result will always return the single value
  4% completed | elapsed time:     10s | time left:    255s  last batch size:      0  2023-09-12 14:00:47,745 - qm - WARNING  - Fetching single result will always return the single value
  5%

<xarray.Dataset>
Dimensions:  (dim_0: 1000)
Coordinates:
    x0       (dim_0) float64 0.01 0.02 0.03 0.04 0.05 ... 0.97 0.98 0.99 1.0
    x1       (dim_0) float64 4.0 4.0 4.0 4.0 4.0 ... 94.0 94.0 94.0 94.0 94.0
Dimensions without coordinates: dim_0
Data variables:
    y0       (dim_0) float64 9.551e-05 9.668e-05 ... 9.332e-05 9.293e-05
Attributes:
    tuid:                             20230912-140034-533-300902
    name:                             avg3
    grid_2d:                          True
    grid_2d_uniformly_spaced:         True
    1d_2_settables_uniformly_spaced:  False
    xlen:                             100
    ylen:                             10